### Osceola CFLRP 
#### Ecoregion generation
First step toward building the Osceola into LANDIS-II is to generate ecoregions -- climatically and edaphically similar areas, wherein soil and climate will provide the same drivers for vegetation growth. The Osceola isn't a huge area (~90k ha) and also is not exceptionally mountainous. This should make creating ecoregions a simple affair. 

The strategy will be to leverage GSSURGO data to describe the major soil types across the landscape, and look for differences in for example water holding capacity. Then we will overlay those differences with crude elevation bands, using elevation as a surrogate for climate -- so long as precipitation and temperature appear to vary across an elevation gradient within the study area.

####GSSURGO data
This always gives me a headache. So here are the steps I used to make sense of the GSSURGO data once its downloaded. I downloaded the data from https://gdg.sc.egov.usda.gov/ on 8-28-2016
1. Load the MapunitRaster into a GIS of your  choice
2. Import two data tables, 'chorizon' and 'component'. I don't understand the naming convention here, chorizon isn't a word, so presumably this stands for c horizon, as the table has a lot of soil horizon data -- clay, sand, silt at various depths etc. However the same logic cannot be applied to 'component', as c omponent doesn't make any sense...
3. Create a spatial join between the MapunitRaster and 'component' using 'MUKEY' from the MapunitRaster (presumably standing for Mapunit Key) and 'Mapunit Key' from 'component'. Why both columns aren't called the same thing, when in fact they represent the same data ...
4. Create a spatial join between MapunitRaster and 'chorizon' using the newly acquired 'Component Key' from MapunitRaster.component. 'chorizon' has the same name for its 'Component Key' column, just to futher add to the confusion via a moment of clarity.
5. Now you have access to the spatial data from 'chorizon' and 'component' on a per MapunitRaster basis.
6. If you want to access the parent group material, useful for splitting up the analysis area into edaphically dissimilar chunks, you need to access the 'copm' table. To make use of it, you need to join it to the MapunitRaster via the 'Component Parent Material Group Key' attribute, which doesn't exist in any of the tables we've joined so far. So, we need an intermediate join that speaks both languages -- namely 'copmgrp'. By joining 'copmgrp' to MapunitRaster via 'Component Key', we can access the parent material columns spatially via our MapunitRaster layer.

The next step isn't entirely intuitive either -- clipping the resulting geodatabase to a smaller vector (e.g., clipping the GSSURGO data for all of FL to just the Osceola extent) requires clipping the 'MUPOLYGON' feature inside the GSSURGO geodatabase. In that manner you end up with a clipped .gdb that retains all associations with the MUKEY and consequently Mapunit Key fields, which you can still relate to soil properties following the steps above.